In [ ]:
import itertools
import pandas as pd
import numpy as np
import plotly.express as px
import skimage
import scipy
import plotly.io as pio
# pio.renderers.default = "jupyterlab"
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import DBSCAN


def bump(x):
    return np.where(np.abs(x) < 1, np.exp(1 / (x ** 2 - 1) + 1) ** 0.2, 0)

In [ ]:
tdse_file = r"D:\DATA\H_TDSE\H-momspe-2d-sw.dat"


def load_data(file, signal_col=4):
    tdse_data = pd.read_csv(file, header=None, sep=r'\s+')
    l = len(tdse_data.columns)

    c = ["p", "theta"] + [""] * (l - 2)
    c[signal_col] = "signal"
    tdse_data.columns = c

    r, theta, signal = tdse_data["p"], tdse_data["theta"], tdse_data["signal"]
    shape = (len(np.unique(r)), len(np.unique(theta)))
    r = np.array(r).reshape(shape)
    theta = np.array(theta).reshape(shape)
    signal = np.array(signal).reshape(shape)
    amins = np.argmin(
            scipy.ndimage.gaussian_filter(signal, sigma=2, mode="wrap"),
            axis=1).flatten()

    x = np.linspace(-0.45, 0.45, 512)
    y = np.linspace(-0.45, 0.45, 512)

    X, Y = np.meshgrid(x, y)
    R, THETA = np.sqrt(X ** 2 + Y ** 2), np.arctan2(Y, X) + np.pi
    R_scaled = R / np.max(r) * shape[0]
    THETA_scaled = THETA / (2 * np.pi) * shape[1]

    signal_resampled = scipy.ndimage.map_coordinates(signal, [R_scaled.flatten(), THETA_scaled.flatten()], order=1,
                                                     mode="wrap").reshape(X.shape)
    signal_resampled *= bump((R - 0.35) / 0.15)
    signal_resampled /= np.sum(signal_resampled)

    rmins, thetamins = np.unique(r), np.unique(theta)[amins]
    thetamins = thetamins % np.pi
    mask = (rmins < 0.4) & (rmins > 0.25)
    rmins = rmins[mask]
    thetamins = thetamins[mask]

    rm = np.linspace(0.27, 0.38, 100)
    tm = scipy.interpolate.make_smoothing_spline(rmins, thetamins, lam=1e-5)(rm)

    # px.imshow(signal_resampled,origin='lower',x=x,y=y).add_scatter(x=rm*np.cos(tm),y=rm*np.sin(tm)).show()

    return signal_resampled, rm, tm, x, y


tdse_signal, tdse_rmins, tdse_thetamins, x, y = load_data(tdse_file)
sfhk_file = r"D:\DATA\H_SFHK\H-wfHK.dat"
sfhk_signal, sfhk_rmins, sfhk_thetamins, x, y = load_data(sfhk_file, signal_col=2)

In [ ]:
def load_vortex_data(file):
    data = pd.read_csv(file, header=None, sep=r'\s+',
                       names=["r", "3", "5", "1", "-1", "phi",
                              "a", "b", "c", "d", "dphi_de"])
    norm = data["3"].max()
    for i in ["3", "5", "1", "-1"]:
        data[i] = data[i].astype(float) / norm

    px.line(x=data.r, y=data["-1"],
            ).add_scatter(
            x=data.r, y=data["1"], name=r"1"
    ).add_scatter(
            x=data.r, y=data["3"], name=r"3"
    ).add_scatter(
            x=data.r, y=data["5"], name=r"5"
    ).show()

    data["phi"] = data["phi"].astype(float)

    data["E"] = data.r ** 2 / 2
    data.drop(columns=["a", "b", "c", "d"], inplace=True)
    data = data.query("0.25<r<0.4")
    return data


vortex_SFHK_file = r"D:\DATA\H_SFHK\H-vortex-diffpha-sfhk.dat"
SFHK_vortex_data = load_vortex_data(vortex_SFHK_file)

vortex_TDSE_file = r"D:\DATA\H_TDSE\H-vortex-diffpha-tdse.dat"
TDSE_vortex_data = load_vortex_data(vortex_TDSE_file)

In [ ]:
fig = make_subplots(
        rows=3, cols=2,
        specs=
        [[{"l": 0, "r": -0.075, }, {"l": -0.075, "r": 0}],
         [{"colspan": 2, "t": 0, "b": -0.02}, None],
         [{"colspan": 2, "secondary_y": True, "t": -0.05}, None]],
        row_heights=[0.315, 0.4, 0.4]

).update_layout(
        width=600, height=1000,
        margin=dict(l=0, r=0, t=0, b=0),
)
fig.layout.template = "seaborn+presentation"
fig.update_xaxes(
        minor_showgrid=True,
        ticks="outside",
        ticklen=6,
)
fig.update_yaxes(
        minor_showgrid=True,
        ticks="outside",
        ticklen=6,
)

fig.add_trace(
        go.Heatmap(
                z=tdse_signal,
                x=x, y=y,
                colorscale="Inferno",
                showscale=False,
        ),
        row=1, col=1
)
fig.add_scatter(row=1, col=1,
                x=tdse_rmins * np.cos(tdse_thetamins),
                y=tdse_rmins * np.sin(tdse_thetamins),
                mode="lines",
                line=dict(color="white", width=2, dash="dot"),
                showlegend=False,
                )

fig.add_trace(
        go.Heatmap(
                z=sfhk_signal,
                x=x, y=y,
                colorscale="Inferno",
                showscale=False,
        ),
        row=1, col=2
)
fig.add_scatter(row=1, col=2,
                x=tdse_rmins * np.cos(tdse_thetamins),
                y=tdse_rmins * np.sin(tdse_thetamins),
                mode="lines",
                line=dict(color="white", width=2, dash="dot"),
                showlegend=False,
                )

# fig.update_xaxes(
#
#         col=1,row=2,
#
#         scaleanchor="x",
# )

fig.update_xaxes(
        title='Major Axis (amu)',
        row=1,
)
fig.update_xaxes(
        row=1,
        col=2,
        scaleanchor="x",
)

fig.update_yaxes(
        title='Minor Axis (amu)',
        row=1, col=1,
        scaleanchor="x",
)
fig.update_yaxes(
        row=1, col=2,
        showticklabels=False,
        scaleanchor="x",
)

line_params = [
    ("-1", "orange", "circle"),
    ("1", "green", "square"),
    ("3", "red", "diamond"),
    ("5", "blue", "cross"),
]
for i in line_params:
    n = " " + i[0] if len(i[0]) == 1 else i[0]

    fig.add_trace(
            go.Scatter(
                    x=SFHK_vortex_data.E * 27.211386246,
                    y=SFHK_vortex_data[i[0]],
                    mode="lines+markers",
                    legendgroup="SFHK",
                    legendgrouptitle_text="SFHK",
                    name=f" l={n} ",
                    marker=dict(size=5, color=i[1], symbol="circle"),
                    line=dict(width=2, color=i[1]),
            ),
            row=2, col=1
    )
    fig.add_trace(
            go.Scatter(
                    x=TDSE_vortex_data.E * 27.211386246,
                    y=TDSE_vortex_data[i[0]],
                    legendgroup="TDSE",
                    legendgrouptitle_text="TDSE",
                    mode="lines+markers",
                    name=f"",
                    marker=dict(size=5, color=i[1], symbol="cross"),
                    line=dict(width=2, color=i[1], dash="dot"),
            ),
            row=2, col=1
    )
fig.update_xaxes(
        row=3, col=1,
        title="Energy (eV)",
)
fig.update_yaxes(
        row=2, col=1,
        title="Probability Density",
        title_text="Population (AU)"
)
fig.update_xaxes(
        row=2, col=1,
        showticklabels=False
)

fig.update_layout(
        legend=dict(
                yanchor="top",
                y=.675,
                xanchor="right",
                x=1,
                font=dict(size=14),
                title_font=dict(size=10),
                grouptitlefont_size=14,
                # bordercolor="lightgrey",
                # borderwidth=1,
                orientation="h",
                entrywidth=30,
                # bgcolor="lightgrey",

        ),
)

fig.add_trace(
        go.Scatter(
                x=SFHK_vortex_data.E * 27.211386246,
                y=SFHK_vortex_data["phi"],
                mode="lines+markers",
                line=dict(width=2, color="red"),
                marker=dict(size=5, color="red", symbol="circle"),
                name=r"$\phi$",
                showlegend=False,
        ),
        row=3, col=1
)
fig.add_trace(
        go.Scatter(
                x=TDSE_vortex_data.E * 27.211386246,
                y=TDSE_vortex_data["phi"],
                mode="lines+markers",
                line=dict(width=2, color="red", dash="dot"),
                marker=dict(size=5, color="red", symbol="cross"),
                name=r"$\phi$",
                showlegend=False,
        ),
        row=3, col=1
)

fig.add_trace(
        go.Scatter(
                x=SFHK_vortex_data.E * 27.211386246,
                y=SFHK_vortex_data["dphi_de"] * 0.02419,
                mode="lines+markers",
                line=dict(width=2, color="blue"),
                marker=dict(size=5, color="blue", symbol="circle"),
                name=r"$\phi$",
                showlegend=False,
        ),
        row=3, col=1, secondary_y=True
)
fig.add_trace(
        go.Scatter(
                x=TDSE_vortex_data.E * 27.211386246,
                y=TDSE_vortex_data["dphi_de"] * 0.02419,
                mode="lines+markers",
                line=dict(width=2, color="blue", dash="dot"),
                marker=dict(size=5, color="blue", symbol="cross"),
                name=r"$\phi$",
                showlegend=False,
        ),
        row=3, col=1, secondary_y=True
)

fig.add_hline(
        row=3, col=1,
        secondary_y=True,
        line=dict(color="black", width=1, dash="dashdot"),
        y=1.334,
        annotation_text="Laser Period",
        annotation_position="top left",

)
fig.update_yaxes(
        row=3, col=1,
        title_text="Phase Difference (rad)",
        range=[-2.2, 2.2],
        tickvals=np.linspace(-2, 2, 5),
        secondary_y=False
)
fig.update_yaxes(
        row=3, col=1,
        title_text="Group Delay (fs)",
        range=[-2.2, 2.2],
        tickvals=np.linspace(-2, 2, 5),
        secondary_y=True
)

fig.add_annotation(
        xref="x4 domain", yref="y4 domain",
        x=0.65, y=0.25,
        text=r"$\Delta \varphi_{3,5} = \varphi_{L=3}- \varphi_{L=5}$",
        showarrow=False,
        font_color="red"
)

fig.add_annotation(
        xref="x4 domain", yref="y4 domain",
        x=0.5, y=0.73,
        text=r"$\tau_g=\frac{d}{dE}\Delta \varphi_{3,5}$",
        showarrow=False,
        font_color="blue"
)

fig.add_annotation(
        text="(a)",
        xref="paper",
        yref="y domain",
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color="white"
)

fig.add_annotation(
        text="(b)",
        xref="paper",
        yref="y domain",
        x=0.540,
        y=0.975,
        showarrow=False,
        font_color="white"
)

fig.add_annotation(
        text="(c)",
        xref="paper",
        yref="y3 domain",
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color="black"
)

fig.add_annotation(
        text="(d)",
        xref="paper",
        yref="y4 domain",
        x=0.025,
        y=0.975,
        showarrow=False,
        font_color="black",
        # font_weight="bold"
)

fig.show()
fig.write_image(
        r"C:\Users\mcman\DataspellProjects\vmi-analysis\Hydrogen Figure.pdf"
)